Этот блокнот показывает, как можно построить модель Cross-Nested Ordered Probit на PyTorch, обучить её на своих данных, и провести оценку значимости коэффициентов. 

Подробности: https://habr.com/ru/post/548100/

Для разминки, накодим обычный Ordered Probit

In [1]:
import torch
import torch.nn as nn


class OrderedProbitHead(nn.Module):
    """ A layer transforming a vector of hidden states into a matrix of probabilities.
    Input size: [batch, 1]
    Output size: [batch, levels]
    """
    def __init__(self, levels):
        super(OrderedProbitHead, self).__init__()
        assert levels >= 3
        self.levels = levels
        self.bias = nn.Parameter(torch.randn(1))
        self.log_difs = nn.Parameter(torch.randn(levels - 2))
        self.activation = torch.distributions.normal.Normal(0, 1).cdf

    @property
    def cutpoints(self):
        diffs = torch.cat([torch.tensor([0]), torch.exp(self.log_difs)], 0)
        return self.bias + torch.cumsum(diffs, 0)
    
    def forward(self, x):
        cdfs = self.activation(self.cutpoints - x)  
        lhs = torch.cat([cdfs, torch.ones_like(x)], 1)
        rhs = torch.cat([torch.zeros_like(x), cdfs], 1)
        return lhs - rhs


class OrderedProbitModel(nn.Module):
    """ A model transforming a vector of features into a matrix of probabilities
    Input size: [batch, features]
    Output size: [batch, levels]
    """
    def __init__(self, features, levels, smoothing=1e-10):
        super(OrderedProbitModel, self).__init__()
        self.levels = levels
        self.dense = nn.Linear(features, 1, bias=False)
        self.head = OrderedProbitHead(levels)
        self.smoothing = smoothing

    def forward(self, x):
        probas = self.head(self.dense(x))
        if self.smoothing:
            probas = (1 - self.smoothing) * probas + self.smoothing * torch.ones_like(probas) / self.levels
        return probas

Проверяем на игрушечных данных и убеждаемся, что все коэффициенты оцениваются верно

In [2]:
import numpy as np

In [3]:
cuts = np.array([-1, 1])
slopes = np.array([1, 0])

np.random.seed(1)
n = 10000
X = np.random.normal(size=[n, 2])
ss = np.dot(X, slopes) + np.random.normal(size=n)
y = (ss[:, np.newaxis] < cuts).sum(1)

In [4]:
xx = torch.tensor(X, dtype=torch.float)
yy = torch.tensor(y, dtype=torch.long)

In [5]:
from tqdm.auto import tqdm, trange

def train(model, x, y, steps=300, lr=0.1, max_norm=1.0, wd=1e-6):
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    pbar = trange(steps)
    for i in pbar:
        optimizer.zero_grad()
        proba = model(x)
        loss = loss_fn(torch.log(proba), y)
        loss.backward()
        nn.utils.clip_grad_norm_(op.parameters(), max_norm)
        pbar.set_description(f"Loss: {loss.item():2.10f}")
        optimizer.step()

In [6]:
op = OrderedProbitModel(features=2, levels=3)
train(op, xx, yy)

In [7]:
for name, param in op.named_parameters():
    print(name, '\t', param.data)
print(op.head.cutpoints)

dense.weight 	 tensor([[-1.0323,  0.0040]])
head.bias 	 tensor([-1.0201])
head.log_difs 	 tensor([0.7024])
tensor([-1.0201,  0.9985], grad_fn=<AddBackward0>)


# Application

In [8]:
import pandas as pd

url = 'https://github.com/avidale/cnop/blob/master/application/rate_change.dta?raw=true'
data = pd.read_stata(url)
data['target'] = data.rate_change.apply(lambda x: int(x * 4 + 2))
print(data.shape)
x = torch.tensor(data[['spread', 'pb', 'houst', 'gdp']].values)
y = torch.tensor(data.target)
data.sample(5)

(210, 7)


,date,rate_change,pb,spread,houst,gdp,target
67,5/18/1992,0.00,-1,0.120,1.21,4.9,2
48,1/31/1991,-0.50,-1,-0.918,1.00,3.3,0
123,11/16/1998,-0.25,-1,-0.542,1.57,3.2,1
139,11/14/2000,0.00,1,-0.394,1.56,5.9,2
62,12/05/1991,-0.25,-1,-0.190,1.02,3.5,1


In [9]:
op = OrderedProbitModel(features=4, levels=5, smoothing=1e-10)
train(op, x, y)
# Loss: 0.7598916888

Коэффициенты получаются такие же, как [в статье](https://www.hse.ru/data/2018/05/30/1149326878/193EC2018.pdf) (на 18 странице)

In [10]:
print(op.dense.weight.data)
# tensor([[0.9262, 1.5742, 1.3730, 0.2391]])
print(op.head.cutpoints.data)
# tensor([0.4655, 1.8380, 4.8357, 6.3309])

tensor([[1.5743, 0.9265, 1.3694, 0.2389]])
tensor([0.4595, 1.8323, 4.8293, 6.3243])


С моделью CNOP чуть сложнее: там функция правдоподобия не является глобально выпуклой, и иногда обучение застревает в плохом локальном оптимуме. Чтобы получить надёжный результат, лучше запустить функцию обучения несколько раз. 

In [13]:
class CrossNestedOrderedProbitModel(nn.Module):
    """ A model transforming a vector of features into a matrix of probabilities.
    The model uses a neutral category (center), 
    negative categories (from 0 to center -1),
    and positive categories (from center + 1 to levels - 1).
    For each group of categories, parameters are different.
    Input size: [batch, features]
    Output size: [batch, levels]
    """
    def __init__(self, features, levels, center, smoothing=1e-10):
        super(CrossNestedOrderedProbitModel, self).__init__()
        self.features = features
        self.levels = levels
        self.center = center
        self.smoothing = smoothing
        self.dense = nn.Linear(features, 3, bias=False)
        self.head_zero = OrderedProbitHead(3)
        self.head_neg = OrderedProbitHead(center + 1)
        self.head_pos = OrderedProbitHead(levels - center)

    def forward(self, x, y=None):
        dense = self.dense(x)
        nzp = self.head_zero(dense[:, [0]])
        negative = self.head_neg(dense[:, [1]])
        positive = self.head_pos(dense[:, [2]])
        probas = torch.zeros([x.shape[0], self.levels])
        probas[:, self.center] += nzp[:, 1]
        probas[:, :(self.center+1)] += nzp[:, [0]] * negative
        probas[:, self.center:] += nzp[:, [2]] * positive
        if self.smoothing:
            probas = (1-self.smoothing) * probas + self.smoothing * torch.ones_like(probas) / self.levels
        return probas
    
cnop = CrossNestedOrderedProbitModel(features=4, levels=5, center=2)
train(cnop, x, y, lr=0.1, steps=10000)
# Loss: 0.6336604357

In [14]:
(cnop(x).argmax(1) == y).numpy().mean()

0.7523809523809524

### 2nd derivative

In [15]:
def get_standard_errors(model, loss):
    all_params = torch.cat([p.view(-1) for p in model.parameters()])
    hessian = torch.empty(all_params.shape * 2)

    first_order_grads = torch.autograd.grad(loss, model.parameters(), retain_graph=True, create_graph=True)

    c = 0
    for i, (name, param) in enumerate(model.named_parameters()):
        v = param.view(-1)
        g = first_order_grads[i].view(-1)
        var = torch.empty_like(v)
        for j, gg in enumerate(g):
            hh = torch.autograd.grad(gg, model.parameters(), retain_graph=True)
            hessian[c] = torch.cat([p.view(-1) for p in hh])
            c += 1

    standard_deviations = torch.diag(torch.inverse(hessian)) ** 0.5

    result = []
    start = 0
    for i, (name, param) in enumerate(model.named_parameters()):
        v = param.view(-1)
        n = v.shape[0]
        ss = standard_deviations[start:start+n].view(param.shape)
        result.append(ss)
        start += n
    return result


In [16]:
from scipy.stats import norm


def report(model, loss):
    result = []
    names = []
    se = get_standard_errors(model, loss)
    for i, (name, param) in enumerate(model.named_parameters()):
        shape = param.squeeze().shape
        v = param.view(-1).detach().numpy()
        s = se[i].view(-1).detach().numpy()
        for j, (vv, ss) in enumerate(zip(v, s)):
            t = np.abs(vv / ss)
            result.append({
                'value': vv,
                'std': ss,
                't': t,
                'p-value': norm.cdf(-t) * 2,
                '[2.5%': vv-ss*1.96,
                '97.5%]': vv+ss*1.96,
            })
            if len(v) > 1:
                names.append(f'{name}{list(np.unravel_index(j, shape))}')
            else:
                names.append(name)
    return pd.DataFrame(result, index=names)


likelihood = nn.CrossEntropyLoss(reduction='sum')
report(op, likelihood(torch.log(op(x)), y)).round(4)

,value,std,t,p-value,[2.5%,97.5%]
dense.weight[0],1.5743,0.1871,8.4159,0.0000,1.2077,1.9410
dense.weight[1],0.9265,0.1479,6.2629,0.0000,0.6365,1.2164
dense.weight[2],1.3694,0.3459,3.9593,0.0001,0.6915,2.0473
dense.weight[3],0.2389,0.0572,4.1779,0.0000,0.1268,0.3510
head.bias,0.4595,0.5382,0.8538,0.3932,-0.5953,1.5143
head.log_difs[0],0.3169,0.1456,2.1770,0.0295,0.0316,0.6022
head.log_difs[1],1.0976,0.0886,12.3879,0.0000,0.9239,1.2713
head.log_difs[2],0.4021,0.1462,2.7508,0.0059,0.1156,0.6886


In [17]:
report(cnop, likelihood(torch.log(cnop(x)), y)).round(4)

,value,std,t,p-value,[2.5%,97.5%]
"dense.weight[0, 0]",2.0720,0.3878,5.3433,0.0000,1.3120,2.8321
"dense.weight[0, 1]",1.3717,0.4164,3.2938,0.0010,0.5554,2.1879
"dense.weight[0, 2]",5.7803,1.1166,5.1767,0.0000,3.5917,7.9688
"dense.weight[0, 3]",0.4283,0.1235,3.4688,0.0005,0.1863,0.6703
"dense.weight[1, 0]",1.0465,0.2939,3.5603,0.0004,0.4704,1.6226
"dense.weight[1, 1]",0.5979,0.3431,1.7427,0.0814,-0.0745,1.2703
"dense.weight[1, 2]",-0.5598,0.7012,0.7984,0.4246,-1.9342,0.8145
"dense.weight[1, 3]",0.1381,0.0762,1.8123,0.0699,-0.0113,0.2875
"dense.weight[2, 0]",3.0320,1.1782,2.5733,0.0101,0.7226,5.3413
"dense.weight[2, 1]",3.6257,1.3942,2.6006,0.0093,0.8932,6.3583
